## Overview

The Gemini File API provides a simple way for developers to upload files and use them with the Gemini API in multimodal scenarios. This notebook shows how to use the File API to upload an image and include it in a `GenerateContent` call to the Gemini API. For more information, refer to the [File API FAQ](https://docs.google.com/document/d/1WBVc5W6PZvgaHLV43UGSrtwHqUmofPT0K0oHuNd7GHA) or the [API documentation](https://ai.google.dev/api/rest/v1beta/files)

Note: This API is currently in beta and is [only available in certain regions](https://ai.google.dev/available_regions).


## Setup


### Install the Gemini API Python SDK

In [ ]:
# Install the Python SDK
!pip install google-generativeai

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Upload a file to the File API

The File API lets you upload a variety of multi-modal MIME types including images. The File API is only intended as input to generate content and has the following attributes:

* Can only be used with [`generateContent`](https://ai.google.dev/api/rest/v1beta/models/generateContent) or [`streamGenerateContent`](https://ai.google.dev/api/rest/v1beta/models/streamGenerateContent) (Chat uses GenerateContent behind the scenes).
* Automatic file deletion after 2 days
* Maximum 2GB per file, 20GB limit per project
* No downloads allowed



First, we will prepare a sample image to upload to the API. We will use a sample image published by the Generative AI SDK.

To upload your own file, see the [Appendix section](#scrollTo=1vDOxWIpHPyD).

In [ ]:
!curl -o image.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

Next, we'll upload that file to the File API.

In [ ]:
sample_file = genai.upload_file(path="image.jpg",
                            display_name="Sample drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

## Get file and list files

`files.get` lets you get the file metadata that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique. Only use the `displayName` to identify files if you manage uniqueness yourself.

In [ ]:
# Get files uploaded in the API
file = genai.get_file(name=sample_file.name)
print(f"Uploaded file '{file.display_name}' as: {sample_file.uri}")

In [ ]:
# Alternatively, list files via the API.
for n, f in zip(range(5), genai.list_files()):
  print(f.uri)

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI. In this example, `makeGenerateContentRequest()` creates a request that starts with a text `Part` and then has a series of image file `Part`s, but valid `Part`s can come in any order.

In [ ]:
# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent LLM call
response = model.generate_content(["Describe the image with a creative description:", sample_file])

Markdown(">" + response.text)

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
# Delete the files with its resource name
genai.delete_file(sample_file.name)
print(f'Deleted {sample_file.display_name}.')

## Appendix: Use your own files by uploading to colab

This notebook useds the Files Api with files that were downloaded from the internet. If you're running thin in Colab and want to use your own files you first need to uplload them to the colab instance.

First, click **Files** on the left sidebar, then click the **Upload** button:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

Next, we'll upload that file to the File API. In the form for the code cell below, enter the filename for the file you uploaded and provide an appropriate display name for the file, then run the cell.


In [ ]:
# Specify the file to upload
my_filename = "gemini_logo.png" # @param {type:"string"}
my_file_display_name = "Gemini Logo" # @param {type:"string"}

my_file = genai.upload_file(path=my_filename,
                            display_name=my_file_display_name)
print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")